### bibliotecas importadas

In [ ]:
import requests
import re
import time
import os
import shutil
import pickle

from bs4 import BeautifulSoup

In [ ]:
class Pokemon:
    def __init__(self, id, name, types):
        self.name=name
        self.id=id
        self.types=types
    def __str__(self):
        return f'{self.id} {self.name} -> {self.types}'

In [27]:
pokeType = ['Water','Electric','Dark','Fairy','Fire','Flying','Ice','Rock','Ground','Steel','Bug','Normal','Grass','Fighting','Poison',' middot ','Ghost','Psychic','Dragon']
games = [
        'gold-silver-crystal',
        'red-blue-yellow',
        'firered-leafgreen',
        'ruby-sapphire-emerald',
        'platinum',
        'heartgold-soulsilver',
        'black-white',
        'black-white-2'
    ]
def extrair_id(texto):
    soup = BeautifulSoup(texto, 'html.parser')
    ids = [element['id'] for element in soup.find_all(id=True)]
    return ids

def extrair_class(texto):
    soup = BeautifulSoup(texto, 'html.parser')
    classes = [css_class for element in soup.find_all(class_=True) for css_class in element["class"]]
    classes = list(set(classes))
    return classes

def extrair_tags_html(texto):
    soup = BeautifulSoup(texto, 'html.parser')
    tags_encontradas = soup.find_all()
    return tags_encontradas

def extrair_conteudo_por_classe_e_id(texto, tag_alvo, classe_alvo=None, id_alvo=None):
    soup = BeautifulSoup(texto, 'html.parser')

    atributos_alvo = {}
    if classe_alvo:
        atributos_alvo['class'] = classe_alvo
    if id_alvo:
        atributos_alvo['id'] = id_alvo

    tag_encontrada = soup.find(tag_alvo, atributos_alvo)

    if tag_encontrada:
        conteudo = tag_encontrada.text
        return conteudo
    else:
        return None 

def get_corret_url(game):
    global games
    url = "https://pokemondb.net/pokedex/game/*"
    
    for index in range(len(games)):
        if game == index+1:
            url = url[0:len(url)-1]
            url += games[index]
    return url

def obter_html(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            return "Not Found"
    except Exception as e:
        print(e)
        
def extrair_conteudo_da_tag(texto, tag_alvo):
    soup = BeautifulSoup(texto, 'html.parser')
    tag_encontrada = soup.find(tag_alvo)
    if tag_encontrada:
        conteudo = tag_encontrada.text
        return conteudo
    else:
        return None 

def catalogar_html(html):
    try:
        soup = BeautifulSoup(html, 'html.parser')
        return soup.prettify()
    except Exception as e:
        print(e)

def set_poke_name(html_string):
    pokeType = ['Water','Electric','Dark','Fairy','Fire','Flying','Ice','Rock','Ground','Steel','Bug','Normal','Grass','Fighting','Poison',' middot ','Ghost','Psychic','Dragon']
    poke_list = list()
    flag=0
    aux = ""
    for i in html_string:
        if flag == 1:
            aux+=i
        if(i == '>' and flag ==0):
            flag=1                
        elif(i == '<' and flag == 1):
            aux = aux.replace('\n','')
            aux="".join([ch for ch in aux if ch.isalpha()])
            if aux not in pokeType:
                poke_list.append(aux)
            aux=''
            flag=0
    poke_list = [item for item in poke_list if item.strip()]
    poke_list.pop()
    poke_list.pop()
    first_poke=['Treecko','Chikorita','Turtwig','Bulbasaur','Victini']
    for i in range(len(poke_list)):
        if poke_list[0] in first_poke:
            
            return poke_list
        else:
            del poke_list[0] 
    return poke_list

def cleardata():
    caminho_diretorio = 'data/'
    if os.path.exists(caminho_diretorio):
        arquivos = os.listdir(caminho_diretorio)
        arquivos = [arquivo for arquivo in arquivos if os.path.isfile(os.path.join(caminho_diretorio, arquivo))]
        quantidade_arquivos = len(arquivos)
        try:
            shutil.rmtree(caminho_diretorio)
            os.makedirs(caminho_diretorio)
            print(f"O diretório '{caminho_diretorio}' foi excluído com sucesso. seu conteudo também, com {quantidade_arquivos} arquivos")
        except Exception as e:
            print(f"Erro ao excluir o diretório: {e}")
    else:
        os.makedirs(caminho_diretorio)
        
def get_info_directory():
    caminho_diretorio = 'data/'
    if os.path.exists(caminho_diretorio):
        arquivos = os.listdir(caminho_diretorio)
        arquivos = [arquivo for arquivo in arquivos if os.path.isfile(os.path.join(caminho_diretorio, arquivo))]
        quantidade_arquivos = len(arquivos)
        return quantidade_arquivos
    else:
        os.makedirs(caminho_diretorio)
    

In [ ]:


cout = 1
print('pokedexes disponíveis:')
for option_game in games:
    print(f'{cout} - {option_game.replace("-"," ")}')
    cout+=1
game = int(input("selecione a pokedex que você deseja:\n"))

url = get_corret_url(game)
if url[len(url)-1] == '*':
    print("Input inválido")
else:
    answer = catalogar_html(obter_html(url))
    lista = set_poke_name(answer)
    print(f'Total: {len(lista)} pokémons')
    for item in lista:
        print(item,end='\n')

In [ ]:
list_name_pokes = set_poke_name(catalogar_html(obter_html(get_corret_url(2))))
quantidade = 1
cleardata()
tempo_inicio = time.time()
for i in range(quantidade):
    with open(f'data/poke{i}.html','w') as arquivo:
        arquivo.write(catalogar_html(obter_html(f'https://pokemondb.net/pokedex/{list_name_pokes[i]}')))
        time.sleep(0.3)
        print(f'{round(((1+i)/quantidade)*100,2)}%...')
tempo_decorrido = time.time() - tempo_inicio
print(f'Terminado! com o tempo de {tempo_decorrido:.5f}')

In [ ]:
quantidade = get_info_directory()
aux = None
for i in range(quantidade):
    with open(f'data/poke{i}.html', 'r') as arquivo:
        linhas = arquivo.readlines()
        aux=''.join(linhas)
#extrair_conteudo_por_classe_e_id(texto, tag_alvo, classe_alvo=None, id_alvo=None)
typePoke=[]
for i in pokeType:
    sup=extrair_conteudo_por_classe_e_id(aux, "a", i.lower(), None)
    if sup != None:
        sup = sup.replace('\n','')
        sup = sup.replace(' ','')
        typePoke.append(sup)
    if len(typePoke)==2:
        break
teste = extrair_conteudo_da_tag(aux, "p")
print(teste)
aux=extrair_conteudo_por_classe_e_id(aux, "main", None, "main")
#extrair_conteudo_da_tag(texto, tag_alvo)

aux = aux.replace('\n','')
aux= aux.split(" ")
name=None
pokeId=None
cout=1
for i in range(len(aux)):
    if len(aux[i]) != 0 and cout==2:
        pokeId=aux[i]
        break
    elif len(aux[i]) != 0 and name == None:
        name=aux[i]
        cout=2
poke = Pokemon(pokeId,name, typePoke)
print(poke)
#cleardata()

In [ ]:
texto_html = "<p>Isso é um exemplo <b>de</b> HTML.</p><a href='#'>Link</a>"
tags_encontradas = extrair_tags_html(texto_html)

for tag in tags_encontradas:
    print(f'Tag encontrada: {tag.name}')

lista_de_tags = [tag.name for tag in tags_encontradas]
print(f'Lista de tags encontradas: {lista_de_tags}')

 

texto_html = "<p>Isso é um exemplo <b>de</b> HTML.</p><a href='#'>Link</a>"
tag_alvo = 'p' 
conteudo_da_tag = extrair_conteudo_da_tag(texto_html, tag_alvo)

if conteudo_da_tag is not None:
    print(f'Conteúdo dentro da tag <{tag_alvo}>: {conteudo_da_tag}')
else:
    print(f'Tag <{tag_alvo}> não encontrada.')
from bs4 import BeautifulSoup

 
    
texto_html = """
<div>
  <p class="paragrafo-exemplo" id="paragrafo1">Isso é um exemplo <b>de</b> HTML.</p>
  <p class="paragrafo-exemplo" id="paragrafo2">Outro parágrafo <a href="#">com link</a>.</p>
</div>
"""

tag_alvo = 'p'
classe_alvo = 'paragrafo-exemplo'
id_alvo = 'paragrafo1'

conteudo_da_tag = extrair_conteudo_por_classe_e_id(texto_html, tag_alvo, classe_alvo, id_alvo)

if conteudo_da_tag is not None:
    print(f'Conteúdo dentro da tag <{tag_alvo} class="{classe_alvo}" id="{id_alvo}">: {conteudo_da_tag}')
else:
    print(f'Tag <{tag_alvo} class="{classe_alvo}" id="{id_alvo}"> não encontrada.')


In [ ]:
quantidade = get_info_directory()
aux = None
for i in range(quantidade):
    with open(f'data/poke{i}.html', 'r') as arquivo:
        linhas = arquivo.readlines()
        aux=''.join(linhas)
#extrair_conteudo_por_classe_e_id(texto, tag_alvo, classe_alvo=None, id_alvo=None)
typePoke=[]
print(aux)
print(
#extrair_conteudo_da_tag(texto, tag_alvo)
#cleardata()

In [30]:
cout = 1
game = 1
url = get_corret_url(game)
answer = catalogar_html(obter_html(url))
print("="*30)
print("ids:")
print(extrair_id(aux))
print("="*30)
print("classes:")
print(extrair_class(aux))
tag_request = input("tag:")
id_request = input("id:")
class_request = input("class:")

soup = BeautifulSoup(answer, 'html.parser')

elemento_id = soup.find(id='meu_id')

conteudo = elemento_id.get_text(separator='\n')  # Use '\n' como separador para manter a estrutura original

print(conteudo)

ids:
['sitesearch', 'main', 'dex-basics', 'tab-basic-1', 'dex-stats', 'bsa-zone_1642018196755-7_123456', 'dex-evolution', 'dex-flavor', 'dex-moves', 'tab-moves-19', 'tab-moves-18', 'dex-sprites', 'dex-locations', 'dex-lang', 'bsa-zone_1612827518798-2_123456']
classes:
['sv-tabs-panel-list', 'main-menu-heading', 'header-logo', 'infocard-arrow', 'ent-name', 'type-fx-200', 'type-electric', 'sitesearch-form', 'num-infinity', 'type-normal', 'main-menu-item-search', 'text-pink', 'img-sprite-v1', 'shining-pearl', 'component', 'type-table', 'sitesearch-input', 'yellow', 'cell-name', 'infocard-lg-data', 'brilliant-diamond', 'main-menu-icon', 'sortwrap', 'infocard-lg-img', 'shield', 'list-focus', 'text-small', 'ultra-sun', 'cell-icon', 'type-ice', 'type-table-pokedex', 'cell-med-text', 'sword', 'text-center', 'span-md-6', 'icon-arrow', 'grid-col', 'wrap-right', 'img-fixed', 'grid-row', 'img-sprite-v13', 'barchart-bar', 'lets-go-pikachu', 'white', 'type-cell', 'black-2', 'infocard', 'grid-contain

tag: 
id: main
class: 


AttributeError: 'NoneType' object has no attribute 'get_text'